# 4.1 Régression Linéaire en Pratique

## PSY3913/6913 - IA, Psychologie et Neuroscience Cognitive

Dans ce notebook, nous allons explorer la régression linéaire de manière pratique :
1. Créer des données synthétiques
2. Implémenter la régression linéaire from scratch
3. Visualiser les résultats
4. Utiliser scikit-learn
5. Appliquer à un exemple de neurosciences

## 1. Import des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Pour des graphiques plus jolis
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Pour la reproductibilité
np.random.seed(42)

## 2. Création de données synthétiques simples

Commençons avec un exemple simple en 1D : prédire y à partir de x.

Nous allons générer des données suivant la relation : **y = 2x + 1 + bruit**

In [ ]:
# Générer des données synthétiques
n_samples = 100
X = np.random.rand(n_samples, 1) * 10  # Valeurs entre 0 et 10
y_true = 2 * X + 1  # Vraie relation linéaire
noise = np.random.randn(n_samples, 1) * 2  # Bruit gaussien
y = y_true + noise  # Données observées

print(f"Forme de X: {X.shape}")
print(f"Forme de y: {y.shape}")
print(f"\nPremiers échantillons:")
print(f"X[:5] = {X[:5].flatten()}")
print(f"y[:5] = {y[:5].flatten()}")

### Visualisation des données

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Données observées')
plt.plot(X, y_true, 'r-', linewidth=2, label='Vraie relation (sans bruit)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Données synthétiques pour la régression linéaire', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Observation: Les points sont dispersés autour de la ligne rouge (vraie relation)")
print("   Le bruit rend les données plus réalistes.")

## 3. Implémentation from scratch : Descente de gradient

Nous allons implémenter la régression linéaire en utilisant la descente de gradient.

**Rappel :**
- Modèle : $\hat{y} = wx + b$
- Loss (MSE) : $L = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
- Gradients : 
  - $\frac{\partial L}{\partial w} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i) x_i$
  - $\frac{\partial L}{\partial b} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)$

In [ ]:
class LinearRegressionGD:
    """
    Régression linéaire avec descente de gradient.
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iterations = n_iterations
        self.w = None
        self.b = None
        self.loss_history = []
        
    def fit(self, X, y):
        """
        Entraîner le modèle.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples, 1)
        """
        n_samples, n_features = X.shape
        
        # Initialisation aléatoire des paramètres
        self.w = np.random.randn(n_features, 1) * 0.01
        self.b = 0
        
        # Descente de gradient
        for i in range(self.n_iterations):
            # Prédiction
            y_pred = self.predict(X)
            
            # Calcul de la loss
            loss = np.mean((y - y_pred) ** 2)
            self.loss_history.append(loss)
            
            # Calcul des gradients
            dw = -(2 / n_samples) * X.T.dot(y - y_pred)
            db = -(2 / n_samples) * np.sum(y - y_pred)
            
            # Mise à jour des paramètres
            self.w -= self.lr * dw
            self.b -= self.lr * db
            
            # Affichage occasionnel
            if (i + 1) % 100 == 0:
                print(f"Itération {i+1}/{self.n_iterations}, Loss: {loss:.4f}")
    
    def predict(self, X):
        """
        Faire des prédictions.
        """
        return X.dot(self.w) + self.b
    
    def get_params(self):
        """
        Retourner les paramètres appris.
        """
        return {'w': self.w, 'b': self.b}

### Entraînement du modèle

In [ ]:
# Créer et entraîner le modèle
model_scratch = LinearRegressionGD(learning_rate=0.01, n_iterations=1000)
print("Entraînement du modèle...\n")
model_scratch.fit(X, y)

# Récupérer les paramètres appris
params = model_scratch.get_params()
print(f"\n📊 Paramètres appris:")
print(f"   w (poids) = {params['w'][0, 0]:.4f}")
print(f"   b (biais) = {params['b']:.4f}")
print(f"\n🎯 Paramètres vrais:")
print(f"   w = 2.0000")
print(f"   b = 1.0000")

### Visualisation de la courbe d'apprentissage

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(model_scratch.loss_history, linewidth=2)
plt.xlabel('Itération', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Courbe d\'apprentissage - Évolution de la loss', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Observation: La loss diminue progressivement et converge.")
print("   C'est un signe que l'apprentissage fonctionne bien!")

### Visualisation des prédictions

In [ ]:
# Prédictions
y_pred_scratch = model_scratch.predict(X)

# Visualisation
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Données observées')
plt.plot(X, y_true, 'r-', linewidth=2, label='Vraie relation', alpha=0.5)
plt.plot(X, y_pred_scratch, 'g-', linewidth=2, label='Prédictions (from scratch)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Régression linéaire - Comparaison', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calcul des métriques
mse_scratch = np.mean((y - y_pred_scratch) ** 2)
r2_scratch = 1 - (np.sum((y - y_pred_scratch) ** 2) / np.sum((y - np.mean(y)) ** 2))

print(f"\n📈 Performance du modèle (from scratch):")
print(f"   MSE = {mse_scratch:.4f}")
print(f"   R² = {r2_scratch:.4f}")

## 4. Utilisation de scikit-learn

Maintenant, comparons avec l'implémentation de scikit-learn qui utilise les équations normales (solution analytique).

In [ ]:
# Créer et entraîner le modèle sklearn
model_sklearn = LinearRegression()
model_sklearn.fit(X, y)

# Récupérer les paramètres
w_sklearn = model_sklearn.coef_[0, 0]
b_sklearn = model_sklearn.intercept_[0]

print(f"📊 Paramètres scikit-learn:")
print(f"   w (poids) = {w_sklearn:.4f}")
print(f"   b (biais) = {b_sklearn:.4f}")

# Prédictions
y_pred_sklearn = model_sklearn.predict(X)

# Métriques
mse_sklearn = mean_squared_error(y, y_pred_sklearn)
r2_sklearn = r2_score(y, y_pred_sklearn)

print(f"\n📈 Performance du modèle (scikit-learn):")
print(f"   MSE = {mse_sklearn:.4f}")
print(f"   R² = {r2_sklearn:.4f}")

### Comparaison des deux approches

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(X, y, alpha=0.5, label='Données observées')
plt.plot(X, y_pred_scratch, 'g-', linewidth=2, label='From scratch (descente de gradient)', alpha=0.7)
plt.plot(X, y_pred_sklearn, 'b--', linewidth=2, label='Scikit-learn (équations normales)', alpha=0.7)
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Comparaison : From scratch vs Scikit-learn', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Observation: Les deux approches donnent des résultats très similaires!")
print("   Les petites différences sont dues à l'approximation de la descente de gradient.")

## 5. Exemple appliqué : Neurosciences

### Problème : Prédire la vitesse de mouvement à partir de l'activité neuronale

Simulons des données réalistes où :
- **Entrées (X)** : Activité de 10 neurones (firing rates en Hz)
- **Sortie (y)** : Vitesse de mouvement (cm/s)

In [ ]:
# Simuler des données de neurosciences
np.random.seed(123)

n_trials = 200  # Nombre d'essais
n_neurons = 10  # Nombre de neurones enregistrés

# Générer l'activité neuronale (firing rates entre 0 et 50 Hz)
X_neuro = np.random.rand(n_trials, n_neurons) * 50

# Simuler la relation : certains neurones contribuent plus que d'autres
# Poids vrais : certains neurones sont plus informatifs
true_weights = np.array([0.5, 0.8, 0.2, 0.1, 0.6, 0.3, 0.7, 0.4, 0.1, 0.5])
true_bias = 10.0

# Calculer la vitesse avec du bruit
y_neuro_true = X_neuro.dot(true_weights.reshape(-1, 1)) + true_bias
noise_neuro = np.random.randn(n_trials, 1) * 5
y_neuro = y_neuro_true + noise_neuro

print(f"📊 Dataset de neurosciences:")
print(f"   Nombre d'essais: {n_trials}")
print(f"   Nombre de neurones: {n_neurons}")
print(f"   Forme de X: {X_neuro.shape}")
print(f"   Forme de y: {y_neuro.shape}")
print(f"\n   Vitesse moyenne: {y_neuro.mean():.2f} cm/s")
print(f"   Vitesse std: {y_neuro.std():.2f} cm/s")

### Division en ensembles d'entraînement et de test

In [ ]:
# Diviser les données : 80% entraînement, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X_neuro, y_neuro, test_size=0.2, random_state=42
)

print(f"📊 Division des données:")
print(f"   Entraînement: {X_train.shape[0]} échantillons")
print(f"   Test: {X_test.shape[0]} échantillons")

### Entraînement du modèle

In [ ]:
# Entraîner le modèle
model_neuro = LinearRegression()
model_neuro.fit(X_train, y_train)

# Prédictions
y_train_pred = model_neuro.predict(X_train)
y_test_pred = model_neuro.predict(X_test)

# Métriques
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"📈 Performance sur l'ensemble d'entraînement:")
print(f"   MSE = {train_mse:.4f}")
print(f"   R² = {train_r2:.4f}")
print(f"\n📈 Performance sur l'ensemble de test:")
print(f"   MSE = {test_mse:.4f}")
print(f"   R² = {test_r2:.4f}")

### Visualisation : Prédictions vs Vraies valeurs

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Ensemble d'entraînement
axes[0].scatter(y_train, y_train_pred, alpha=0.6)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
             'r--', linewidth=2, label='Ligne parfaite (y=ŷ)')
axes[0].set_xlabel('Vitesse réelle (cm/s)', fontsize=12)
axes[0].set_ylabel('Vitesse prédite (cm/s)', fontsize=12)
axes[0].set_title(f'Ensemble d\'entraînement\nR² = {train_r2:.3f}', fontsize=13)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Ensemble de test
axes[1].scatter(y_test, y_test_pred, alpha=0.6, color='green')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Ligne parfaite (y=ŷ)')
axes[1].set_xlabel('Vitesse réelle (cm/s)', fontsize=12)
axes[1].set_ylabel('Vitesse prédite (cm/s)', fontsize=12)
axes[1].set_title(f'Ensemble de test\nR² = {test_r2:.3f}', fontsize=13)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observation: Les points sont bien alignés sur la ligne y=ŷ,")
print("   ce qui indique que le modèle fait de bonnes prédictions!")

### Analyse des poids : Quels neurones sont importants ?

In [ ]:
# Récupérer les poids appris
learned_weights = model_neuro.coef_[0]

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Comparaison poids vrais vs appris
x_pos = np.arange(n_neurons)
width = 0.35

axes[0].bar(x_pos - width/2, true_weights, width, label='Poids vrais', alpha=0.8)
axes[0].bar(x_pos + width/2, learned_weights, width, label='Poids appris', alpha=0.8)
axes[0].set_xlabel('Neurone', fontsize=12)
axes[0].set_ylabel('Poids', fontsize=12)
axes[0].set_title('Comparaison des poids', fontsize=13)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels([f'N{i+1}' for i in range(n_neurons)])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Importance relative (valeur absolue)
importance = np.abs(learned_weights)
sorted_idx = np.argsort(importance)[::-1]

axes[1].barh(range(n_neurons), importance[sorted_idx], alpha=0.8)
axes[1].set_yticks(range(n_neurons))
axes[1].set_yticklabels([f'Neurone {i+1}' for i in sorted_idx])
axes[1].set_xlabel('Importance (|poids|)', fontsize=12)
axes[1].set_title('Importance relative des neurones', fontsize=13)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print(f"\n🧠 Analyse des neurones:")
print(f"\n   Top 3 neurones les plus informatifs:")
for i, idx in enumerate(sorted_idx[:3]):
    print(f"   {i+1}. Neurone {idx+1}: poids = {learned_weights[idx]:.3f}")

### Analyse des résidus (erreurs)

In [ ]:
# Calculer les résidus
residuals_train = y_train - y_train_pred
residuals_test = y_test - y_test_pred

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Résidus vs Prédictions (entraînement)
axes[0, 0].scatter(y_train_pred, residuals_train, alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Prédictions', fontsize=11)
axes[0, 0].set_ylabel('Résidus', fontsize=11)
axes[0, 0].set_title('Résidus vs Prédictions (entraînement)', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

# Résidus vs Prédictions (test)
axes[0, 1].scatter(y_test_pred, residuals_test, alpha=0.6, color='green')
axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Prédictions', fontsize=11)
axes[0, 1].set_ylabel('Résidus', fontsize=11)
axes[0, 1].set_title('Résidus vs Prédictions (test)', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

# Distribution des résidus (entraînement)
axes[1, 0].hist(residuals_train, bins=20, alpha=0.7, edgecolor='black')
axes[1, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Résidus', fontsize=11)
axes[1, 0].set_ylabel('Fréquence', fontsize=11)
axes[1, 0].set_title('Distribution des résidus (entraînement)', fontsize=12)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Distribution des résidus (test)
axes[1, 1].hist(residuals_test, bins=15, alpha=0.7, color='green', edgecolor='black')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Résidus', fontsize=11)
axes[1, 1].set_ylabel('Fréquence', fontsize=11)
axes[1, 1].set_title('Distribution des résidus (test)', fontsize=12)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("   1. Les résidus sont distribués de manière aléatoire autour de 0")
print("   2. Pas de pattern visible → le modèle linéaire est approprié")
print("   3. Distribution approximativement gaussienne → hypothèses respectées")

## 6. Exercice pratique

### À vous de jouer ! 

**Exercice :** Modifiez le code ci-dessus pour :
1. Changer le nombre de neurones (essayez avec 5 ou 20 neurones)
2. Augmenter le bruit dans les données
3. Observer comment la performance (R²) change

**Questions de réflexion :**
- Que se passe-t-il si vous avez très peu de données d'entraînement ?
- Comment la performance change-t-elle avec plus de neurones ?
- Que se passe-t-il si certains neurones ne sont pas informatifs (poids = 0) ?

In [ ]:
# Votre code ici
# ...


## 7. Conclusion

Dans ce notebook, nous avons :

✅ Implémenté la régression linéaire from scratch avec la descente de gradient

✅ Utilisé scikit-learn pour une implémentation robuste

✅ Appliqué la régression à un problème de neurosciences

✅ Appris à évaluer et visualiser les performances

✅ Interprété les poids pour comprendre l'importance des features

### Points clés à retenir:

1. **La régression linéaire** est un modèle simple mais puissant pour prédire des valeurs continues
2. **L'évaluation** doit se faire sur un ensemble de test séparé
3. **R²** mesure la proportion de variance expliquée (0 à 1, plus c'est élevé, mieux c'est)
4. **Les résidus** doivent être distribués aléatoirement autour de 0
5. **Les poids** indiquent l'importance relative des features

### Prochaine étape:
Passez au notebook **4.2_classification_lineaire.ipynb** pour explorer la classification linéaire!